# Questioning Barbie and Oppenheimer Through the Use of Agents

In the following notebook we will build an application that queries both the Barbie and Oppenheimer movies Wikipedia pages, as well as their reviews. 

The main focus of this notebook is to showcase a brief introduction to Agents.

## Build 🏗️

There are 3 main tasks in this notebook:

1. Contruct a Barbie retriever
2. Construct an Oppenheimer retriever
3. Combine the two and allow users to query both resources from a single input through the use of Agents

## Ship 🚢

Based on Tuesday's session - construct a Chainlit (or Gradio) application that allows users to interface with the application.

## Share 🚀

Make a social media post about your final application.

### Dependencies

As always, let's start with some dependencies!

In [1]:
!pip install -q -U langchain openai

In [2]:
import getpass
import os
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

### LLM 

We will be leveraging OpenAI's `gpt-3.5-turbo` throughout the notebook, and we can keep it consistent throughout!

In [4]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0)

### Data Collection and Transformation

We'll be leveraging the `WikipediaLoader` tool to collect information from Wikipedia. 

Be sure to set the `doc_content_chars_max` parameter so that you capture the *entire* Wikipedia article content.

In [5]:
!pip install -q -U wikipedia

In [8]:
from langchain.document_loaders import WikipediaLoader, CSVLoader

barbie_wikipedia_docs = WikipediaLoader(
    query="Barbie (film)", 
    load_max_docs= 200, # YOUR CODE HERE, 
    doc_content_chars_max= 10000### YOUR CODE HERE
    ).load()

barbie_csv_docs = CSVLoader(
    file_path= "./barbie_data/barbie.csv", ### YOUR CODE HERE, 
    source_column= "Review"### YOUR CODE HERE
    ).load()

/Users/daweizhang/anaconda3/envs/mlops/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/daweizhang/anaconda3/envs/mlops/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [15]:
barbie_wikipedia_docs[0]

Document(page_content='Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of self-discovery following an existential crisis. It features an ensemble supporting cast, including America Ferrera, Kate McKinnon, Helen Mirren, Issa Rae, Simu Liu, Michael Cera, Rhea Perlman, and Will Ferrell.\nA live-action Barbie film was announced in September 2009 by Universal Pictures with Laurence Mark producing. Development began in April 2014, when Sony Pictures acquired the film rights. Following multiple writer and director changes and the casting of Amy Schumer and later Anne Hathaway as Barbie, the rights were transferred to Warner Bros. Pictures in October 2018. Robbie was cast in 2019, after Gal Gadot turned down the r

In [16]:
barbie_csv_docs[0]

Document(page_content=': 0\nReview_Date: 21 July 2023\nAuthor: LoveofLegacy\nRating: 6\nReview_Title: Beautiful film, but so preachy\nReview: Margot does the best with what she\'s given, but this film was very disappointing to me. It was marketed as a fun, quirky satire with homages to other movies. It started that way, but ended with over-dramatized speeches and an ending that clearly tried to make the audience feel something, but left everyone just feeling confused. And before you say I\'m a crotchety old man, I\'m a woman in my 20s, so I\'m pretty sure I\'m this movie\'s target audience. The saddest part is there were parents with their kids in the theater that were victims of the poor marketing, because this is not a kid\'s movie. Overall, the humor was fun on occasion and the film is beautiful to look at, but the whole concept falls apart in the second half of the film and becomes a pity party for the "strong" woman.\nReview_Url: /review/rw9199947/?ref_=tt_urv', metadata={'source'

Since we'll be using same format source documentation separated by topic, we can save ourselves some extra effort and set up our splitters once. 

We're going to leverage the `RecursiveCharacterTextSplitter` again, this time paying close attention to the format our Wikipedia articles and reviews are in so we can be sure to chunk them appropritately. 

> HINT: You can pass a list of separators when you intialize your `RecursiveTextSplitter`! They are acted on in order of element 0 -> element len(list).

RELEVANT DOCS:
- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#langchain.text_splitter.RecursiveCharacterTextSplitter)

In [50]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

wikipedia_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,### YOUR CODE HERE,
    chunk_overlap = 60,### YOUR CODE HERE,
    length_function = len,### YOUR CODE HERE,
    is_separator_regex= False,
    separators = ["==", "===", '.', "\n\n", "\n"]### YOUR CODE HERE # keep headings, then paragraphs, then sentences
)

csv_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,### YOUR CODE HERE,
    chunk_overlap = 20,### YOUR CODE HERE,
    length_function = len,### YOUR CODE HERE,
    is_separator_regex= False,
    separators = ["\n", "."] ### YOUR CODE HERE # keep paragraphs, then sentences
)

chunked_barbie_wikipedia_docs = wikipedia_text_splitter.split_documents(barbie_wikipedia_docs)### YOUR CODE HERE
chunked_barbie_csv_docs = csv_text_splitter.split_documents(barbie_csv_docs)### YOUR CODE HERE

In [51]:
chunked_barbie_wikipedia_docs

[Document(page_content='Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach', metadata={'title': 'Barbie (film)', 'summary': 'Barbie is a 2023 American fantasy comedy film directed by Greta Gerwig from a screenplay she wrote with Noah Baumbach. Based on the Barbie fashion dolls by Mattel, it is the first live-action Barbie film after numerous computer-animated films and specials. The film follows Barbie (Margot Robbie) and Ken (Ryan Gosling) on a journey of self-discovery following an existential crisis. It features an ensemble supporting cast, including America Ferrera, Kate McKinnon, Helen Mirren, Issa Rae, Simu Liu, Michael Cera, Rhea Perlman, and Will Ferrell.\nA live-action Barbie film was announced in September 2009 by Universal Pictures with Laurence Mark producing. Development began in April 2014, when Sony Pictures acquired the film rights. Following multiple writer and director changes and the casting of Amy Sc

#### Retrieval and Embedding Strategy

We've already discussed the useful application of `CacheBackedEmbeddings`, so let's do it again!

RELEVANT DOCS:
- [`CacheBackedEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.cache.CacheBackedEmbeddings.html#langchain-embeddings-cache-cachebackedembeddings)

In [52]:
!pip install -q -U rank_bm25 tiktoken faiss-cpu

In [53]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# set up cached embeddings store
store = LocalFileStore("./cache/")### YOUR CODE HERE

core_embeddings_model = OpenAIEmbeddings()### YOUR CODE HERE

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model,
    store,
    namespace=core_embeddings_model.model
)### YOUR CODE HERE

We'll implement a `FAISS` vectorstore, and create a retriever from it.

In [54]:
barbie_csv_faiss_retriever = FAISS.from_documents(chunked_barbie_csv_docs, embedder).as_retriever()### YOUR CODE HERE


There are a number of excellent options to retrieve documents - we'll be looking at an additional example today, which is called the `EnsembleRetriever`.

The method this is using is outlined in [this paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf).

The brief explanation is:

1. We collect results from two different retrieval methods over the same corpus
2. We apply a reranking algorithm to rerank our source documents to be the *most relevant* without losing specific or potentially low-ranked information rich documents
3. We feed the top-k results into the LLM with our query as context.

> HINT: Your weight list should be of type `List[float]` and the `sum(List[float])` should be `1`.

In [55]:
# set up BM25 retriever
barbie_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    ### YOUR CODE HERE
    chunked_barbie_wikipedia_docs
)
barbie_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
barbie_wikipedia_faiss_store = FAISS.from_documents(
    ### YOUR CODE HERE,
    ### YOUR CODE HERE
    chunked_barbie_wikipedia_docs,
    embedder
)
barbie_wikipedia_faiss_retriever = barbie_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
barbie_ensemble_retriever = EnsembleRetriever(
    retrievers=[barbie_wikipedia_bm25_retriever, barbie_wikipedia_faiss_retriever, barbie_csv_faiss_retriever],### YOUR CODE HERE,
    weights= [0.3, 0.4, 0.3]### YOUR CODE HERE # should sum to 1
)

#### Retrieval Agent

We can create a simple conversational retrieval Agent by using the built-ins provided by LangChain!

> HINT: Be sure to provide good natural language descriptions of what the tool should be used for to get the best results.

RELEVANT DOCS:
- [`create_retriever_tool`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool)

In [56]:
from langchain.agents.agent_toolkits import create_retriever_tool

barbie_wikipedia_retrieval_tool = create_retriever_tool(
    ### YOUR CODE HERE 
    retriever=barbie_ensemble_retriever,
    ### YOUR CODE HERE
    name="search_wikipedia_of_barbie",
    ### YOUR CODE HERE
    description="Search the Wikipedia of Barbie for information about Barbie."
)

barbie_csv_retrieval_tool = create_retriever_tool(
    ### YOUR CODE HERE
    retriever=barbie_csv_faiss_retriever,
    ### YOUR CODE HERE
    name="search_reviews_of_barbie",
    ### YOUR CODE HERE
    description="Search reviews of Barbie for information about Barbie."
)

barbie_retriever_tools = [barbie_wikipedia_retrieval_tool, barbie_csv_retrieval_tool]### YOUR CODE HERE

Now that we've created our tools, we can combined them into an agent!

RELEVANT DOCS:
- [`create_conversational_retrieval_agent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent.html#langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent)

In [57]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

barbie_retriever_agent_executor = create_conversational_retrieval_agent(llm, barbie_retriever_tools, verbose=True)# YOUR CODE HERE

In [58]:
barbie_retriever_agent_executor({"input" : "Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?"})



> Entering new AgentExecutor chain...

Invoking: `search_reviews_of_barbie` with `Barbie`


[Document(page_content='. The perfect Barbie.', metadata={'source': "When I sat down to watch this film I assumed it would be some fun pink silly movie but it was not in the best possible way. It starts very much in that way but the second Barbie gets to the real world everything changes and the film makes so many important statements on sexism and the treatment of women. I thought it perfectly addressed the affect Barbie has had on the world and never shied away from it. This movie does not flinch away from the reality of the world or what damage ignorance can do (Ken). America Ferrera's speech is amazing and moved me so much as did Barbie's own found freedom. Robbie was an amazing Barbie and all the women shined in this movie...and Alan was pretty good too. It was everything you can want from a movie, sharp, whitty, moving, honest and progressive. The perfect Barbie.", 'row': 88}), Document(

{'input': 'Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_reviews_of_barbie', 'arguments': '{\n  "__arg1": "Barbie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=\'. The perfect Barbie.\', metadata={\'source\': "When I sat down to watch this film I assumed it would be some fun pink silly movie but it was not in the best possible way. It starts very much in that way but the second Barbie gets to the real world everything changes and the film makes so many important statements on sexism and the treatment of women. I thought it perfectly addressed the affect Barbie has had on the world and never shied awa

In [59]:
barbie_retriever_agent_executor({"input" : "What is a very quick summary of the plot of the Barbie movie?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia_of_barbie` with `Barbie`


[Document(page_content='. The toy is the figurehead of the Barbie brand that includes a range of fashion dolls and accessories', metadata={'title': 'Barbie', 'summary': 'Barbie is a fashion doll created by American businesswoman Ruth Handler, manufactured by American toy company Mattel and launched in 1959. The toy is the figurehead of the Barbie brand that includes a range of fashion dolls and accessories. Barbie has been an important part of the toy fashion doll market for over six decades. Mattel has sold over a billion Barbie dolls, making it the company\'s largest and most profitable line. The brand has expanded into a multimedia franchise since the late 1980s, including video games, computer-animated films, and a live-action film.\nBarbie and her male counterpart, Ken, have been described as the two most popular dolls in the world. Since its launch, Barbie has transformed the toy busin

{'input': 'What is a very quick summary of the plot of the Barbie movie?',
 'chat_history': [AIMessage(content='Based on the reviews I found, it seems that people generally liked the Barbie movie and appreciated its philosophical elements. The movie is described as making important statements on sexism and the treatment of women. It addresses the impact Barbie has had on the world and does not shy away from the reality of the world or the damage caused by ignorance. The movie is praised for being sharp, witty, moving, honest, and progressive.\n\nOne review mentions that Barbie goes through an existential crisis that makes her question Barbie Land and the truth of her existence. Another review describes the movie as a darkly satirical look at gender structure in society.\n\nOverall, it seems that the Barbie movie incorporates philosophical themes and explores deeper issues beyond its surface appearance as a fun pink silly movie.', additional_kwargs={}, example=False),
  HumanMessage(con

### Oppenheimer Retrieval System

We're going to repourpose some of what we created previously, but this time we'll explore a different multi-source retrieval system.

In [60]:
oppenheimer_wikipedia_docs = WikipediaLoader(
    query="Oppenheimer (film)", 
    load_max_docs= 200,
    doc_content_chars_max= 10000
    ).load()### YOUR CODE HERE

oppenheimer_csv_docs = CSVLoader(
    file_path= "./oppenheimer_data/oppenheimer.csv",
    source_column= "Review"
    ).load()### YOUR CODE HERE

/Users/daweizhang/anaconda3/envs/mlops/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/daweizhang/anaconda3/envs/mlops/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [61]:
chunked_opp_wikipedia_docs = wikipedia_text_splitter.split_documents(oppenheimer_wikipedia_docs)### YOUR CODE HERE
chunked_opp_csv_docs = csv_text_splitter.split_documents(oppenheimer_csv_docs)### YOUR CODE HERE

In [62]:
store = LocalFileStore("./oppcache")
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model,
    store,
    namespace=core_embeddings_model.model
)
opp_csv_faiss_retriever = FAISS.from_documents(chunked_opp_csv_docs, embedder).as_retriever()### YOUR CODE HERE

# set up BM25 retriever
opp_wikipedia_bm25_retriever = BM25Retriever.from_documents(chunked_opp_wikipedia_docs)### YOUR CODE HERE
opp_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
opp_wikipedia_faiss_store = FAISS.from_documents(chunked_opp_wikipedia_docs, embedder)### YOUR CODE HERE
opp_wikipedia_faiss_retriever = opp_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
opp_ensemble_retriever = EnsembleRetriever(
    retrievers=[opp_csv_faiss_retriever, opp_wikipedia_bm25_retriever, opp_wikipedia_faiss_retriever],
    weights= [0.3, 0.4, 0.3]
)### YOUR CODE HERE

#### Multi-source chain

We're going to allow the LLM to decide which information is most -> least valuable.

The way we'll do this is with LangChain's rather powerful "Expression Language"!

> HINT: You can leverage [this](https://python.langchain.com/docs/use_cases/question_answering/how_to/multiple_retrieval) resource if you get stuck - but experiment with different prompts/formats.

In [63]:
from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: public user reviews about the Oppenheimer movie
<source1>
{source1}
</source1>

Source 2: the wikipedia page for the Oppenheimer movie including the plot summary, cast, and production information
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{question}")])

In [64]:
oppenheimer_multisource_chain = {
    "source1": (lambda x: x["question"]) | opp_ensemble_retriever,
    "source2": (lambda x: x["question"]) | opp_csv_faiss_retriever,
    "question": lambda x: x["question"],
} | prompt | llm

In [65]:
oppenheimer_multisource_chain.invoke({"question" : "What did people think of the Oppenheimer movie?"})

AIMessage(content="People had mixed opinions about the Oppenheimer movie. Some reviewers praised the film, describing it as a thought-provoking and visually stunning biography. They appreciated Christopher Nolan's visionary direction, Cillian Murphy's portrayal of J. Robert Oppenheimer, and the film's breathtaking cinematography. These reviewers found the storytelling to be intricate and the performances of the supporting cast to be outstanding. They believed that Oppenheimer was a triumph that left a lasting impact and showcased the power of cinema to enlighten and inspire.\n\nHowever, there were also negative reviews. One reviewer mentioned that they wanted the journey of the movie to be more smooth and sharply written. Another reviewer questioned whether Christopher Nolan could do justice to the subject of J. Robert Oppenheimer. Additionally, there were two people who left the full-house screening of the movie, indicating that it may not have appealed to everyone.\n\nOverall, opinio

# Agent Creation

Now we can finally start building our Agent!

The first thing we'll need to do is provide our Agent a Toolbelt. (list of tools). Much like Batman, our LLM-powered Agent can use these tools as it sees fit. 

While the examples we're constructing in this notebook are straightforward for brevity and simplicities sake - there is no limit to what you can build with Agents, as we'll see as we progress through the program.

So, let's begin by setting up our Tools!

You'll notice that we have to set up a function to allow our `OppenheimerInfo` tool to interface with the Agent - this is due to it have a specific required input. Creating custom tools is a pattern that you'll want to grow acustomed to as you use LangChain more and more.

In [66]:
from langchain.agents import Tool

def query_oppenheimer(input):
    return oppenheimer_multisource_chain.invoke({"question" : input})

tools = [
    Tool(
        name = "SearchBarbie",### YOUR CODE HERE,
        func=barbie_retriever_agent_executor.invoke,### YOUR CODE HERE,
        description="Search info about Barbie."### YOUR CODE HERE
    ),
    Tool(
        name = "SearchOppenheimer",### YOUR CODE HERE,
        func= query_oppenheimer,### YOUR CODE HERE,
        description="Search info about Oppenheimer." ### YOUR CODE HERE
    ),
]

Now that we've set up our Agents toolbelt, let's set up the LLM that will be powering it!

I would suggest playing around with these prompts - and experiments to find what works best for you.

RELEVANT DOCS:
- [`ZeroShotAgent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.mrkl.base.ZeroShotAgent.html#langchain-agents-mrkl-base-zeroshotagent)

In [68]:
from langchain.agents import ZeroShotAgent, AgentExecutor

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    ### YOUR CODE HERE
    tools=tools,
    ### YOUR CODE HERE
    prefix=prefix,
    ### YOUR CODE HERE
    suffix=suffix,
    ### YOUR CODE HERE
    input_variables=["input", "agent_scratchpad"]
)

In [69]:
from langchain import LLMChain

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    )### YOUR CODE HERE

All that's left to do now is create our `ZeroShotAgent` and our `AgentExecutor`, which are the "reasoner" and "actor" halfs of the `ReAct` method of Agent implementation.

Read all about the `ReAct` framework [here](https://react-lm.github.io/)

In [70]:
barbenheimer_agent = ZeroShotAgent(
    llm_chain=llm_chain,### YOUR AGENTS, 
    tools=tools,### YOUR AGENTS, 
    verbose=True### YOUR AGENTS
    )

barbenheimer_agent_chain = AgentExecutor.from_agent_and_tools(
    agent=barbenheimer_agent,### YOUR AGENTS, 
    tools=tools,### YOUR AGENTS, 
    verbose=True### YOUR AGENTS
    )

## Conclusion

All that is left to do now, is feed inputs to your Agent and watch it work!

Remember to use the `{"input" : "YOUR QUERY HERE"}` format when prompting the Agent.

In [71]:
barbenheimer_agent_chain.invoke({"input" : "What did people like about the Barbie movie?"})



> Entering new AgentExecutor chain...
Thought: I don't have any information about the Barbie movie, so I should search for it.
Action: SearchBarbie
Action Input: "Barbie movie"

> Entering new AgentExecutor chain...

Invoking: `search_reviews_of_barbie` with `Barbie movie`


[Document(page_content='Review: "Barbie" is a comedy film based on the line of fashion dolls by Mattel', metadata={'source': '"Barbie" is a comedy film based on the line of fashion dolls by Mattel. Directed and co-written by Greta Gerwig ("Lady Bird", "Little Women") and starring Margot Robbie and Ryan Gosling, it takes every opportunity to revel in its amusing premise with a darkly satirical look at gender structure in society.', 'row': 97}), Document(page_content='Review: This movie is specially for those who loves the character of barbie specially in childhood', metadata={'source': "This movie is specially for those who loves the character of barbie specially in childhood. This film is a treat for Barbie lover

{'input': 'What did people like about the Barbie movie?',
 'output': 'People liked the Barbie movie for its amusing premise, satirical look at gender structure, exploration of deeper themes, and feminist representation.'}

In [72]:
barbenheimer_agent_chain.run({"input" : "What did people like about the Oppenheimer movie?"})



> Entering new AgentExecutor chain...
Thought: I don't have any information about the Oppenheimer movie.
Action: SearchOppenheimer
Action Input: Oppenheimer movie
Observation: content='The Oppenheimer movie is a 2023 epic biographical thriller film written and directed by Christopher Nolan. It is based on the 2005 biography "American Prometheus" by Kai Bird and Martin J. Sherwin. The film chronicles the career of American theoretical physicist J. Robert Oppenheimer, focusing on his studies, his leadership of the Manhattan Project during World War II, and his subsequent fall from grace due to a 1954 security hearing.\n\nThe film stars Cillian Murphy as J. Robert Oppenheimer, Emily Blunt as his wife "Kitty", Matt Damon as head of the Manhattan Project Leslie Groves, Robert Downey Jr. as U.S. Atomic Energy Commission member Lewis Strauss, and Florence Pugh as Communist Party USA member Jean Tatlock. The ensemble supporting cast includes Josh Hartnett, Casey Affleck, Rami Malek, and Kenn

'People liked the Oppenheimer movie for its cast performances, screenplay, and visuals.'

In [73]:
barbenheimer_agent_chain.run({"input" : "Did the movies Barbie and Oppenheimer share similar themes or ideas?"})



> Entering new AgentExecutor chain...
Thought: I need to find information about both Barbie and Oppenheimer to determine if their movies share similar themes or ideas.
Action: SearchBarbie
Action Input: "Barbie movies themes and ideas"

> Entering new AgentExecutor chain...

Invoking: `search_wikipedia_of_barbie` with `Barbie`


[Document(page_content='. The toy is the figurehead of the Barbie brand that includes a range of fashion dolls and accessories', metadata={'title': 'Barbie', 'summary': 'Barbie is a fashion doll created by American businesswoman Ruth Handler, manufactured by American toy company Mattel and launched in 1959. The toy is the figurehead of the Barbie brand that includes a range of fashion dolls and accessories. Barbie has been an important part of the toy fashion doll market for over six decades. Mattel has sold over a billion Barbie dolls, making it the company\'s largest and most profitable line. The brand has expanded into a multimedia franchise since the late

'The Barbie movies explore themes such as female empowerment, friendship and relationships, self-expression and individuality, dreams and ambitions, social issues and values'

## Next Steps

It's time to build a Chainlit (or Gradio) application and host it on Hugging Face Spaces! :ship: